In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import shapiro
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
import itertools
import os
from datetime import datetime
from scipy import stats
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
import catboost
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb



# Disables all deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Carregando os datasets para o treinamento e concantenando-os
df = pd.read_csv("../data/train.csv")
df =df.drop(columns='id')
# Preenchendo Weight Capacity (kg) com média
df['Weight Capacity (kg)'] = df['Weight Capacity (kg)'].fillna(df['Weight Capacity (kg)'].mean())

# Selecionar as colunas do tipo 'object'
categorical_columns = df.select_dtypes(include=['object'])

# Filtrar as colunas que têm valores faltantes (NaN)
columns_with_nan = categorical_columns.columns[categorical_columns.isna().any()]

# Criar uma lista com o nome das colunas e o número de valores ausentes
columns_nan_count = [(column, df[column].isna().sum()) for column in columns_with_nan]

# Ordenar a lista pelo número de valores ausentes (de forma crescente)
columns_nan_count_sorted = sorted(columns_nan_count, key=lambda x: x[1])

# Extrair somente os nomes das colunas, agora ordenados
sorted_columns = [column for column, _ in columns_nan_count_sorted]
print(sorted_columns)

['Size', 'Waterproof', 'Laptop Compartment', 'Style', 'Material', 'Brand', 'Color']


In [3]:
for column in sorted_columns:
    
    # Preparing the dataset
    df_with_target = df[df[column].notna()]
    df_with_nan_target = df[df[column].isna()]
        
    # Label Encoder
    le = LabelEncoder()
    df_with_target[column] = le.fit_transform(df_with_target[column])

    # Removing columns with NaN values
    df_with_target_without_nan_columns = df_with_target.dropna(axis=1, how='any')
    df_with_nan_target_without_nan_columns = df_with_nan_target.dropna(axis=1, how='any')
        
    # Checking for categorical values in df_with_nan_target_without_nan_columns
    columns_to_onehotencode = df_with_nan_target_without_nan_columns.select_dtypes(include='object').columns
    
    if len(columns_to_onehotencode) > 0:
        df_with_target_without_nan_columns = pd.get_dummies(df_with_target_without_nan_columns, columns= columns_to_onehotencode, drop_first=False)
        df_with_nan_target_without_nan_columns = pd.get_dummies(df_with_nan_target_without_nan_columns, columns=columns_to_onehotencode, drop_first=False)
    
    
    # Splitting features and targets
    X = df_with_target_without_nan_columns.drop(columns=column)
    y = df_with_target_without_nan_columns[column]
    
    # Model training
    model = RandomForestClassifier()
    model.fit(X, y)
    
    # Predciting missing values
    y_pred = model.predict(df_with_nan_target_without_nan_columns)
    
    # Replacing missing values in original dataframe by predicted values
    df.loc[df[column].isna(), column] = y_pred

C:\Users\julia\AppData\Local\Temp\ipykernel_16896\2448118777.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])
C:\Users\julia\AppData\Local\Temp\ipykernel_16896\2448118777.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])
C:\Users\julia\AppData\Local\Temp\ipykernel_16896\2448118777.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [4]:
for column in sorted_columns:
    
    # Preparando o dataset
    df_with_target = df[df[column].notna()]
    df_with_nan_target = df[df[column].isna()]
        
    # Label Encoder
    le = LabelEncoder()
    df_with_target[column] = le.fit_transform(df_with_target[column])

    # Removendo colunas com valores NaN
    df_with_target_without_nan_columns = df_with_target.dropna(axis=1, how='any')
    df_with_nan_target_without_nan_columns = df_with_nan_target.dropna(axis=1, how='any')
        
    # Verificando variáveis categóricas em df_with_nan_target_without_nan_columns
    columns_to_onehotencode = df_with_nan_target_without_nan_columns.select_dtypes(include='object').columns
    
    if len(columns_to_onehotencode) > 0:
        df_with_target_without_nan_columns = pd.get_dummies(df_with_target_without_nan_columns, columns=columns_to_onehotencode, drop_first=False)
        df_with_nan_target_without_nan_columns = pd.get_dummies(df_with_nan_target_without_nan_columns, columns=columns_to_onehotencode, drop_first=False)
    
    # Separando variáveis independentes e dependentes
    X = df_with_target_without_nan_columns.drop(columns=column)
    y = df_with_target_without_nan_columns[column]
    
    # Treinamento do modelo - substituímos RandomForestClassifier por KNeighborsClassifier
    model = AdaBoostClassifier(n_estimators=50, random_state=42)
    model.fit(X, y)
    

    # Predizendo os valores ausentes
    y_pred = model.predict(df_with_nan_target_without_nan_columns)
    
    # Substituindo os valores ausentes no dataframe original pelos valores previstos
    df.loc[df[column].isna(), column] = y_pred


TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['int', 'str']

In [ ]:
df.isna().sum()

Brand                   0
Material                0
Size                    0
Compartments            0
Laptop Compartment      0
Waterproof              0
Style                   0
Color                   0
Weight Capacity (kg)    0
Price                   0
dtype: int64

In [ ]:
# Engenharia de atributos: get dummies para TODAS variáveis categóricas
df_eng = df.copy()
cat_columns = df_eng.select_dtypes(include=['object']).columns
for column in cat_columns:
    df_eng = pd.get_dummies(df_eng, columns=[column], prefix=['dum'], dtype=int)

scaler = MinMaxScaler()
df_eng['Weight Capacity (kg)'] = scaler.fit_transform(df_eng[['Weight Capacity (kg)']])
df_eng['Compartments'] = scaler.fit_transform(df_eng[['Compartments']])

In [ ]:
X = df_eng.drop(columns='Price')
y = df_eng['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ridge = Ridge(alpha=100)
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)

# Calculando o RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"{rmse:.5f}")

38.50513


### COMEÇANDO NOVAMENTE E UTILIZANDO O ADABOOST PARA PREENCHER OS VALORES NAN

In [ ]:
# Carregando os datasets para o treinamento e concantenando-os
df_1 = pd.read_csv("../data/train.csv")
df_2 = pd.read_csv("../data/training_extra.csv")
df = pd.concat([df_1, df_2], axis=0)

#df = pd.read_csv("../data/train.csv")
df =df.drop(columns='id')

# Selecionar as variáveis categóricas em ordem crescente de número de valores faltantes
# Selecionar as colunas do tipo 'object'
categorical_columns = df.select_dtypes(include=['object'])
# Filtrar as colunas que têm valores faltantes (NaN)
columns_with_nan = categorical_columns.columns[categorical_columns.isna().any()]
# Criar uma lista com o nome das colunas e o número de valores ausentes
columns_nan_count = [(column, df[column].isna().sum()) for column in columns_with_nan]
# Ordenar a lista pelo número de valores ausentes (de forma crescente)
columns_nan_count_sorted = sorted(columns_nan_count, key=lambda x: x[1])
# Extrair somente os nomes das colunas, agora ordenados
sorted_columns = [column for column, _ in columns_nan_count_sorted]

# Preenchendo Weight Capacity (kg) com média
df['Weight Capacity (kg)'] = df['Weight Capacity (kg)'].fillna(df['Weight Capacity (kg)'].mean())

# Preenhcendo valores NaN com AdaBoost
for column in sorted_columns:
    
    # Preparando o dataset
    df_with_target = df[df[column].notna()]
    df_with_nan_target = df[df[column].isna()]
        
    # Label Encoder
    le = LabelEncoder()
    df_with_target[column] = le.fit_transform(df_with_target[column])

    # Removendo colunas com valores NaN
    df_with_target_without_nan_columns = df_with_target.dropna(axis=1, how='any')
    df_with_nan_target_without_nan_columns = df_with_nan_target.dropna(axis=1, how='any')
        
    # Verificando variáveis categóricas em df_with_nan_target_without_nan_columns
    columns_to_onehotencode = df_with_nan_target_without_nan_columns.select_dtypes(include='object').columns
    
    if len(columns_to_onehotencode) > 0:
        df_with_target_without_nan_columns = pd.get_dummies(df_with_target_without_nan_columns, columns=columns_to_onehotencode, drop_first=False)
        df_with_nan_target_without_nan_columns = pd.get_dummies(df_with_nan_target_without_nan_columns, columns=columns_to_onehotencode, drop_first=False)
    
    # Separando variáveis independentes e dependentes
    X = df_with_target_without_nan_columns.drop(columns=column)
    y = df_with_target_without_nan_columns[column]
    
    # Treinamento do modelo - substituímos RandomForestClassifier por KNeighborsClassifier
    model = catboost.CatBoostClassifier(iterations=100, depth=10, learning_rate=0.1, random_seed=42)
    model.fit(X, y)
    

    # Predizendo os valores ausentes
    y_pred = model.predict(df_with_nan_target_without_nan_columns)
    
    # Convertendo as previsões numéricas de volta para as categorias originais
    y_pred_original = le.inverse_transform(y_pred)

    # Substituindo os valores ausentes no dataframe original pelos valores previstos
    df.loc[df[column].isna(), column] = y_pred_original

# Engenharia de atributos: get dummies para TODAS variáveis categóricas
df_eng = df.copy()
cat_columns = df_eng.select_dtypes(include=['object']).columns
for column in cat_columns:
    df_eng = pd.get_dummies(df_eng, columns=[column], prefix=['dum'], dtype=int)
# Engenharia de Atributos: normalização dos dados numéricos
scaler = MinMaxScaler()
df_eng['Weight Capacity (kg)'] = scaler.fit_transform(df_eng[['Weight Capacity (kg)']])
df_eng['Compartments'] = scaler.fit_transform(df_eng[['Compartments']])

# Treinamento do modelo
X = df_eng.drop(columns='Price')
y = df_eng['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ridge = Ridge(alpha=100)
ridge.fit(X_train, y_train)

# Predição do target
y_pred = ridge.predict(X_test)

# Calculando o RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"{rmse:.5f}")

C:\Users\julia\AppData\Local\Temp\ipykernel_14684\3308919401.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])


0:	learn: 1.0984199	total: 915ms	remaining: 1m 30s
1:	learn: 1.0982601	total: 1.91s	remaining: 1m 33s
2:	learn: 1.0981332	total: 2.86s	remaining: 1m 32s
3:	learn: 1.0980315	total: 3.33s	remaining: 1m 19s
4:	learn: 1.0979459	total: 4.24s	remaining: 1m 20s
5:	learn: 1.0978734	total: 5.19s	remaining: 1m 21s
6:	learn: 1.0978145	total: 6.17s	remaining: 1m 21s
7:	learn: 1.0977597	total: 7.13s	remaining: 1m 22s
8:	learn: 1.0977196	total: 7.62s	remaining: 1m 17s
9:	learn: 1.0976848	total: 8.61s	remaining: 1m 17s
10:	learn: 1.0976548	total: 9.6s	remaining: 1m 17s
11:	learn: 1.0976272	total: 10.6s	remaining: 1m 17s
12:	learn: 1.0976032	total: 11.5s	remaining: 1m 17s
13:	learn: 1.0975805	total: 12.5s	remaining: 1m 17s
14:	learn: 1.0975605	total: 13.5s	remaining: 1m 16s
15:	learn: 1.0975424	total: 14.4s	remaining: 1m 15s
16:	learn: 1.0975309	total: 15.1s	remaining: 1m 13s
17:	learn: 1.0975141	total: 16s	remaining: 1m 12s
18:	learn: 1.0975026	total: 16.9s	remaining: 1m 12s
19:	learn: 1.0974931	tota

C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\julia\AppData\Local\Temp\ipykernel_14684\3308919401.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])


0:	learn: 0.6930425	total: 207ms	remaining: 20.5s
1:	learn: 0.6930944	total: 414ms	remaining: 20.3s
2:	learn: 0.6930780	total: 626ms	remaining: 20.2s
3:	learn: 0.6930429	total: 829ms	remaining: 19.9s
4:	learn: 0.6930444	total: 1.04s	remaining: 19.8s
5:	learn: 0.6930324	total: 1.24s	remaining: 19.5s
6:	learn: 0.6930115	total: 1.44s	remaining: 19.2s
7:	learn: 0.6929954	total: 1.65s	remaining: 18.9s
8:	learn: 0.6929827	total: 1.86s	remaining: 18.8s
9:	learn: 0.6929767	total: 2.08s	remaining: 18.7s
10:	learn: 0.6929716	total: 2.28s	remaining: 18.4s
11:	learn: 0.6929642	total: 2.49s	remaining: 18.3s
12:	learn: 0.6929519	total: 2.7s	remaining: 18s
13:	learn: 0.6929466	total: 2.9s	remaining: 17.8s
14:	learn: 0.6929408	total: 3.1s	remaining: 17.6s
15:	learn: 0.6929355	total: 3.31s	remaining: 17.4s
16:	learn: 0.6929260	total: 3.53s	remaining: 17.2s
17:	learn: 0.6929201	total: 3.73s	remaining: 17s
18:	learn: 0.6929153	total: 3.94s	remaining: 16.8s
19:	learn: 0.6929068	total: 4.14s	remaining: 16.

C:\Users\julia\AppData\Local\Temp\ipykernel_14684\3308919401.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])


...

0:	learn: 1.6092880	total: 1.6s	remaining: 2m 38s
1:	learn: 1.6091574	total: 3.27s	remaining: 2m 40s
2:	learn: 1.6090415	total: 4.9s	remaining: 2m 38s
3:	learn: 1.6089218	total: 6.54s	remaining: 2m 36s
4:	learn: 1.6088239	total: 8.19s	remaining: 2m 35s
5:	learn: 1.6087263	total: 9.79s	remaining: 2m 33s
6:	learn: 1.6086334	total: 11.4s	remaining: 2m 31s
7:	learn: 1.6085647	total: 13.1s	remaining: 2m 30s
8:	learn: 1.6084805	total: 14.7s	remaining: 2m 28s
9:	learn: 1.6084023	total: 16.3s	remaining: 2m 27s
10:	learn: 1.6083556	total: 18s	remaining: 2m 25s
11:	learn: 1.6082831	total: 19.6s	remaining: 2m 24s
12:	learn: 1.6082207	total: 21.3s	remaining: 2m 22s
13:	learn: 1.6081579	total: 22.9s	remaining: 2m 20s
14:	learn: 1.6080973	total: 24.4s	remaining: 2m 18s
15:	learn: 1.6080514	total: 26.1s	remaining: 2m 16s
16:	learn: 1.6080053	total: 27.7s	remaining: 2m 15s
17:	learn: 1.6079500	total: 29.3s	remaining: 2m 13s
18:	learn: 1.6079074	total: 30.8s	remaining: 2m 11s
19:	learn: 1.6078495	total

C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\julia\AppData\Local\Temp\ipykernel_14684\3308919401.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])


0:	learn: 1.7915113	total: 1.88s	remaining: 3m 6s
1:	learn: 1.7912465	total: 4.03s	remaining: 3m 17s
2:	learn: 1.7910836	total: 5.93s	remaining: 3m 11s
3:	learn: 1.7909031	total: 7.83s	remaining: 3m 7s
4:	learn: 1.7907573	total: 9.8s	remaining: 3m 6s
5:	learn: 1.7906095	total: 11.7s	remaining: 3m 3s
6:	learn: 1.7904815	total: 13.7s	remaining: 3m 1s
7:	learn: 1.7903935	total: 15.5s	remaining: 2m 58s
8:	learn: 1.7903565	total: 16s	remaining: 2m 42s
9:	learn: 1.7902653	total: 18s	remaining: 2m 41s
10:	learn: 1.7901615	total: 19.9s	remaining: 2m 40s
11:	learn: 1.7900910	total: 21.7s	remaining: 2m 39s
12:	learn: 1.7900157	total: 23.6s	remaining: 2m 38s
13:	learn: 1.7899063	total: 25.6s	remaining: 2m 36s
14:	learn: 1.7898981	total: 26.1s	remaining: 2m 27s
15:	learn: 1.7898373	total: 27.9s	remaining: 2m 26s
16:	learn: 1.7897575	total: 29.9s	remaining: 2m 25s
17:	learn: 1.7897015	total: 31.8s	remaining: 2m 24s
18:	learn: 1.7896573	total: 33.7s	remaining: 2m 23s
19:	learn: 1.7895922	total: 35.7

C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


38.86780


### FUNÇÃO DE PREAPARAÇÃO

In [ ]:
def preparation(df):
    # Selecionar as variáveis categóricas em ordem crescente de número de valores faltantes
    # Selecionar as colunas do tipo 'object'
    categorical_columns = df.select_dtypes(include=['object'])
    # Filtrar as colunas que têm valores faltantes (NaN)
    columns_with_nan = categorical_columns.columns[categorical_columns.isna().any()]
    # Criar uma lista com o nome das colunas e o número de valores ausentes
    columns_nan_count = [(column, df[column].isna().sum()) for column in columns_with_nan]
    # Ordenar a lista pelo número de valores ausentes (de forma crescente)
    columns_nan_count_sorted = sorted(columns_nan_count, key=lambda x: x[1])
    # Extrair somente os nomes das colunas, agora ordenados
    sorted_columns = [column for column, _ in columns_nan_count_sorted]

    # Preenchendo Weight Capacity (kg) com média
    df['Weight Capacity (kg)'] = df['Weight Capacity (kg)'].fillna(df['Weight Capacity (kg)'].mean())

    # Preenhcendo valores NaN com AdaBoost
    for column in sorted_columns:
        
        # Preparando o dataset
        df_with_target = df[df[column].notna()]
        df_with_nan_target = df[df[column].isna()]
            
        # Label Encoder
        le = LabelEncoder()
        df_with_target[column] = le.fit_transform(df_with_target[column])

        # Removendo colunas com valores NaN
        df_with_target_without_nan_columns = df_with_target.dropna(axis=1, how='any')
        df_with_nan_target_without_nan_columns = df_with_nan_target.dropna(axis=1, how='any')
            
        # Verificando variáveis categóricas em df_with_nan_target_without_nan_columns
        columns_to_onehotencode = df_with_nan_target_without_nan_columns.select_dtypes(include='object').columns
        
        if len(columns_to_onehotencode) > 0:
            df_with_target_without_nan_columns = pd.get_dummies(df_with_target_without_nan_columns, columns=columns_to_onehotencode, drop_first=False)
            df_with_nan_target_without_nan_columns = pd.get_dummies(df_with_nan_target_without_nan_columns, columns=columns_to_onehotencode, drop_first=False)
        
        # Separando variáveis independentes e dependentes
        X = df_with_target_without_nan_columns.drop(columns=column)
        y = df_with_target_without_nan_columns[column]
        
        # Treinamento do modelo - substituímos RandomForestClassifier por KNeighborsClassifier
        model = GaussianNB()
        model.fit(X, y)
        

        # Predizendo os valores ausentes
        y_pred = model.predict(df_with_nan_target_without_nan_columns)
        
        # Substituindo os valores ausentes no dataframe original pelos valores previstos
        df.loc[df[column].isna(), column] = y_pred

    # Engenharia de atributos: get dummies para TODAS variáveis categóricas
    df_eng = df.copy()
    cat_columns = df_eng.select_dtypes(include=['object']).columns
    for column in cat_columns:
        df_eng = pd.get_dummies(df_eng, columns=[column], prefix=['dum'], dtype=int)
    # Engenharia de Atributos: normalização dos dados numéricos
    scaler = MinMaxScaler()
    df_eng['Weight Capacity (kg)'] = scaler.fit_transform(df_eng[['Weight Capacity (kg)']])
    df_eng['Compartments'] = scaler.fit_transform(df_eng[['Compartments']])

    return df_eng

In [ ]:
def csv_create(y_pred, ids):
      
    # Creating the submission DataFrame with 'id' and 'Transported'
    submission_data = pd.DataFrame({
        'id': ids,
        'Price': y_pred
    })
    
    # Checking for the 'submission' folder
    current_directory = os.getcwd()
    all_items = os.listdir(current_directory)
    folders = [item for item in all_items if os.path.isdir(os.path.join(current_directory, item))]
    
    # Checking if the 'submissions' folder exists, if not, create it
    if 'submissions' not in folders:
        submission_folder = os.path.join(current_directory, 'submissions')
        os.makedirs(submission_folder)
        
    # Getting the current date and time to create a unique filename
    now = datetime.now()
    filename = now.strftime("submission_%d_%m_%y_%H_%M.csv")
    
    # Saving the CSV in the 'submissions' folder with the date and time in the filename
    submission_data.to_csv(f'submissions/{filename}', index=False)
    
    print(f"File '{filename}' has been created in the 'submissions' folder!")

In [ ]:
# Carregando os datasets para o treinamento e concantenando-os
df_1 = pd.read_csv("../data/train.csv")
df_2 = pd.read_csv("../data/training_extra.csv")
df = pd.concat([df_1, df_2], axis=0)
df =df.drop(columns='id')

df_prep = preparation(df)

# Treinamento do modelo
X = df_prep.drop(columns='Price')
y = df_prep['Price']

ridge = Ridge(alpha=100)
ridge.fit(X, y)

C:\Users\julia\AppData\Local\Temp\ipykernel_14684\503016808.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])
C:\Users\julia\AppData\Local\Temp\ipykernel_14684\503016808.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])
C:\Users\julia\AppData\Local\Temp\ipykernel_14684\503016808.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

Ridge(alpha=100)

In [ ]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")
df_train = df_train.drop(columns='id')
df_test = df_test.drop(columns='id')
df_train_prep = preparation(df_train)

C:\Users\julia\AppData\Local\Temp\ipykernel_14684\503016808.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])
C:\Users\julia\AppData\Local\Temp\ipykernel_14684\503016808.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])
C:\Users\julia\AppData\Local\Temp\ipykernel_14684\503016808.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

Index(['Compartments', 'Weight Capacity (kg)', 'Price', 'dum_0', 'dum_1',
       'dum_2', 'dum_3', 'dum_4', 'dum_Adidas', 'dum_Jansport', 'dum_Nike',
       'dum_Puma', 'dum_Under Armour', 'dum_1', 'dum_3', 'dum_Canvas',
       'dum_Leather', 'dum_Nylon', 'dum_Polyester', 'dum_0', 'dum_1',
       'dum_Large', 'dum_Medium', 'dum_Small', 'dum_0', 'dum_1', 'dum_No',
       'dum_Yes', 'dum_0', 'dum_1', 'dum_No', 'dum_Yes', 'dum_1', 'dum_2',
       'dum_Backpack', 'dum_Messenger', 'dum_Tote', 'dum_2', 'dum_4',
       'dum_Black', 'dum_Blue', 'dum_Gray', 'dum_Green', 'dum_Pink',
       'dum_Red'],
      dtype='object')

In [ ]:
def preparation(data):
    df = data.copy()
    # Selecionar as variáveis categóricas em ordem crescente de número de valores faltantes
    # Selecionar as colunas do tipo 'object'
    categorical_columns = df.select_dtypes(include=['object'])
    # Filtrar as colunas que têm valores faltantes (NaN)
    columns_with_nan = categorical_columns.columns[categorical_columns.isna().any()]
    # Criar uma lista com o nome das colunas e o número de valores ausentes
    columns_nan_count = [(column, df[column].isna().sum()) for column in columns_with_nan]
    # Ordenar a lista pelo número de valores ausentes (de forma crescente)
    columns_nan_count_sorted = sorted(columns_nan_count, key=lambda x: x[1])
    # Extrair somente os nomes das colunas, agora ordenados
    sorted_columns = [column for column, _ in columns_nan_count_sorted]
    
    
    # Preenchendo Weight Capacity (kg) com média
    df['Weight Capacity (kg)'] = df['Weight Capacity (kg)'].fillna(df['Weight Capacity (kg)'].mean())
    
    # Preenhcendo valores NaN com AdaBoost
    for column in sorted_columns:
        
        # Preparando o dataset
        df_with_target = df[df[column].notna()]
        df_with_nan_target = df[df[column].isna()]
            
        # Label Encoder
        le = LabelEncoder()
        df_with_target[column] = le.fit_transform(df_with_target[column])

        # Removendo colunas com valores NaN
        df_with_target_without_nan_columns = df_with_target.dropna(axis=1, how='any')
        df_with_nan_target_without_nan_columns = df_with_nan_target.dropna(axis=1, how='any')
            
        # Verificando variáveis categóricas em df_with_nan_target_without_nan_columns
        columns_to_onehotencode = df_with_nan_target_without_nan_columns.select_dtypes(include='object').columns
        
        if len(columns_to_onehotencode) > 0:
            df_with_target_without_nan_columns = pd.get_dummies(df_with_target_without_nan_columns, columns=columns_to_onehotencode, drop_first=False)
            df_with_nan_target_without_nan_columns = pd.get_dummies(df_with_nan_target_without_nan_columns, columns=columns_to_onehotencode, drop_first=False)
        
        # Separando variáveis independentes e dependentes
        X = df_with_target_without_nan_columns.drop(columns=column)
        y = df_with_target_without_nan_columns[column]
        
        # Treinamento do modelo - substituímos RandomForestClassifier por KNeighborsClassifier
        model = AdaBoostClassifier(n_estimators=50, random_state=42)
        model.fit(X, y)
        

        # Predizendo os valores ausentes
        y_pred = model.predict(df_with_nan_target_without_nan_columns)
        
        # Convertendo as previsões numéricas de volta para as categorias originais
        y_pred_original = le.inverse_transform(y_pred)

        # Substituindo os valores ausentes no dataframe original pelos valores previstos
        df.loc[df[column].isna(), column] = y_pred_original
    
    # Engenharia de atributos: get dummies para TODAS variáveis categóricas
    df_eng = df.copy()
    cat_columns = df_eng.select_dtypes(include=['object']).columns
    print(cat_columns)
    
    for column in cat_columns:
        df_eng = pd.get_dummies(df_eng, columns=[column], dtype=int)
    
    
    # Engenharia de Atributos: normalização dos dados numéricos
    scaler = MinMaxScaler()
    df_eng['Weight Capacity (kg)'] = scaler.fit_transform(df_eng[['Weight Capacity (kg)']])
    df_eng['Compartments'] = scaler.fit_transform(df_eng[['Compartments']])
    
    return df_eng

In [ ]:
# Carregando os datasets para o treinamento e concantenando-os
df_1 = pd.read_csv("../data/train.csv")
df_2 = pd.read_csv("../data/training_extra.csv")
df_train = pd.concat([df_1, df_2], axis=0)
df_train =df_train.drop(columns='id')

# Tratando o dataset de treino
df_train_prep = preparation(df_train)

# Separando em base de treino e teste
X = df_train_prep.drop(columns='Price')
y = df_train_prep['Price']

# Treinando o modelo
ridge = Ridge(alpha=100)
ridge.fit(X, y)

# Carregando o dataset de teste
df_test = pd.read_csv("../data/test.csv")

# Guardando os 'ids'
ids = df_test['id']

# Dropando a coluna 'id' para as predições
df_test = df_test.drop(columns='id')

# Tratando o dataset de teste
df_test_prep = preparation(df_test)

# Prevendo os resultados
predictions = ridge.predict(df_test_prep)

# Criando o arquvio de submissão
csv_create(predictions, ids)

C:\Users\julia\AppData\Local\Temp\ipykernel_14684\1251817090.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])
C:\Users\julia\AppData\Local\Temp\ipykernel_14684\1251817090.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])
C:\Users\julia\AppData\Local\Temp\ipykernel_14684\1251817090.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

Index(['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color'],
      dtype='object')


C:\Users\julia\AppData\Local\Temp\ipykernel_14684\1251817090.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])
C:\Users\julia\AppData\Local\Temp\ipykernel_14684\1251817090.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])
C:\Users\julia\AppData\Local\Temp\ipykernel_14684\1251817090.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

Index(['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color'],
      dtype='object')
File 'submission_19_02_25_14_08.csv' has been created in the 'submissions' folder!
